In [ ]:
import pandas as pd
import datetime as dt
from datetime import timedelta

<b>First Dataset</b>

calendar.csv               - Contains information about the dates on which the products are sold.

sales_train_validation.csv - Contains the historical daily unit sales data per product and store [d_1 - d_1913]

sample_submission.csv      - The correct format for submissions. Reference the Evaluation tab for more info.

sell_prices.csv            - Contains information about the price of the products sold per store and date.

sales_train_evaluation.csv - Includes sales (d_1 - d_1941) (labels used for the Public leaderboard)

In [ ]:
calendar_data = pd.read_csv('walmart_sales_data/calendar.csv')
#print(calendar_data.dtypes)
print('\nMinimum date: ', calendar_data['date']. min())
print('Maximum date: ', calendar_data['date']. max())
print('\n',calendar_data.head())


Minimum date:  2011-01-29
Maximum date:  2016-06-19

          date  wm_yr_wk    weekday  wday  month  year    d event_name_1  \
0  2011-01-29     11101   Saturday     1      1  2011  d_1          NaN   
1  2011-01-30     11101     Sunday     2      1  2011  d_2          NaN   
2  2011-01-31     11101     Monday     3      1  2011  d_3          NaN   
3  2011-02-01     11101    Tuesday     4      2  2011  d_4          NaN   
4  2011-02-02     11101  Wednesday     5      2  2011  d_5          NaN   

  event_type_1 event_name_2 event_type_2  snap_CA  snap_TX  snap_WI  
0          NaN          NaN          NaN        0        0        0  
1          NaN          NaN          NaN        0        0        0  
2          NaN          NaN          NaN        0        0        0  
3          NaN          NaN          NaN        1        1        0  
4          NaN          NaN          NaN        1        0        1  


## Google trends

In [ ]:
# Re-normalize the data to obtain daily trends
def normalize_monthly_to_daily(trends_monthly, trends_daily, kw):
    
    trends_daily = trends_daily.rename(columns={kw: f'{kw}_daily'})
    
    trends_monthly = trends_monthly.resample('1D').min().fillna(method='pad')
    trends_monthly = trends_monthly.rename(columns={kw: f'{kw}_monthly'})
    kw_trends_monthly = trends_monthly[f'{kw}_monthly']
    
    merged_daily_trends = pd.concat([trends_daily, kw_trends_monthly], join='inner', axis = 1)
    merged_daily_trends[f'{kw}'] =  merged_daily_trends[f'{kw}_daily'] * merged_daily_trends[f'{kw}_monthly'] / 100
    merged_daily_trends.drop(columns=[f'{kw}_monthly', f'{kw}_daily'], inplace=True)
    return merged_daily_trends

In [ ]:
state_list = ['ca', 'tx', 'wi']
kw_list = ['walmart', 'hobbies', 'household', 'foods']
trends_monthly = pd.read_csv('google_trends_monthly.csv')
trends_monthly.set_index(pd.DatetimeIndex(trends_monthly['date']), inplace=True)
trends_monthly.drop(columns=["date"], inplace=True)
trends_df = pd.DataFrame()
for state in state_list:
    for kw in kw_list:
        trends_daily = pd.read_csv(f'trends_{state}_{kw}_2022-04-25.csv')
        trends_daily.set_index(pd.DatetimeIndex(trends_daily['date']), inplace=True)
        trends_daily.drop(columns=["date"], inplace=True)
        print(trends_daily)
        normalized_trends = normalize_monthly_to_daily(trends_monthly, trends_daily, kw)
        normalized_trends = normalized_trends.rename(columns={kw: f'{state}_{kw}'})
        trends_df = trends_df.join(normalized_trends, how="outer")

trends_df

            walmart
date               
2011-01-01      100
2011-01-02       86
2011-01-03       75
2011-01-04       68
2011-01-05       74
...             ...
2016-06-26       97
2016-06-27       77
2016-06-28       82
2016-06-29       81
2016-06-30       81

[2008 rows x 1 columns]
            hobbies
date               
2011-01-01       75
2011-01-02      100
2011-01-03       70
2011-01-04       28
2011-01-05       63
...             ...
2016-06-26       47
2016-06-27       74
2016-06-28       77
2016-06-29       74
2016-06-30       49

[2008 rows x 1 columns]
            household
date                 
2011-01-01         34
2011-01-02         53
2011-01-03         73
2011-01-04         75
2011-01-05         74
...               ...
2016-06-26         65
2016-06-27         86
2016-06-28         91
2016-06-29         91
2016-06-30         90

[2008 rows x 1 columns]
            foods
date             
2011-01-01     99
2011-01-02    100
2011-01-03     90
2011-01-04     81
2011-01-05 

,ca_walmart,ca_hobbies,ca_household,ca_foods,tx_walmart,tx_hobbies,tx_household,tx_foods,wi_walmart,wi_hobbies,wi_household,wi_foods
date,,,,,,,,,,,,
2011-01-01,50.00,28.50,17.00,69.30,50.00,25.08,14.00,61.60,46.00,10.64,0.00,59.50
2011-01-02,43.00,38.00,26.50,70.00,43.00,38.00,25.50,58.10,50.00,0.00,10.50,69.30
2011-01-03,37.50,26.60,36.50,63.00,40.00,22.42,18.00,69.30,36.50,11.40,15.50,70.00
2011-01-04,34.00,10.64,37.50,56.70,35.00,17.10,32.50,61.60,34.00,0.00,29.50,69.30
2011-01-05,37.00,23.94,37.00,64.40,35.00,28.88,29.00,64.40,32.50,5.32,14.50,58.10
...,...,...,...,...,...,...,...,...,...,...,...,...
2016-05-28,32.40,33.63,33.15,45.92,36.00,34.20,58.50,44.24,31.32,0.00,51.35,47.04
2016-05-29,32.76,57.00,25.35,48.72,36.00,13.11,65.00,46.48,30.60,57.00,17.55,53.76
2016-05-30,36.00,43.32,36.40,54.32,34.92,31.92,52.00,48.16,36.00,10.83,25.35,54.88


In [ ]:
# Merge google trends into calendar_data

# Change the date columns to DateTimeIndex
calendar_data['date'] = pd.to_datetime(calendar_data['date'], format="%Y/%m/%d")
calendar_data.set_index(pd.DatetimeIndex(calendar_data['date']), inplace=True)
calendar_data.drop(columns=['date'], inplace=True)

In [ ]:
# Resample from monthly to daily and join dataframes

calendar_data_expanded = pd.concat([calendar_data, trends_df], join='inner', axis = 1)

calendar_data_expanded.head()

,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,...,ca_household,ca_foods,tx_walmart,tx_hobbies,tx_household,tx_foods,wi_walmart,wi_hobbies,wi_household,wi_foods
date,,,,,,,,,,,,,,,,,,,,,
2011-01-29,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,NaN,...,36.5,63.00,44.00,17.48,33.0,49.70,33.00,0.00,39.0,63.70
2011-01-30,11101,Sunday,2,1,2011,d_2,NaN,NaN,NaN,NaN,...,29.0,67.90,41.50,11.40,34.5,58.80,49.00,20.90,19.0,70.00
2011-01-31,11101,Monday,3,1,2011,d_3,NaN,NaN,NaN,NaN,...,48.0,60.90,36.50,24.70,28.5,56.70,36.00,14.44,6.5,63.00
2011-02-01,11101,Tuesday,4,2,2011,d_4,NaN,NaN,NaN,NaN,...,38.5,62.70,41.34,13.32,55.0,66.00,42.93,7.92,25.3,42.24
2011-02-02,11101,Wednesday,5,2,2011,d_5,NaN,NaN,NaN,NaN,...,35.2,57.42,39.75,11.16,50.6,52.14,45.58,8.64,11.0,48.18


In [ ]:
import plotly.express as px
fig = px.line(calendar_data_expanded, x=calendar_data_expanded.index, y="ca_foods", title='Google trends for "foods" in CA')
fig.show()

In [ ]:
calendar_data_expanded.to_csv('calendar_data_expanded.csv')